In [1]:
%store -r sheyenne_grasslands_gdf caddo_grasslands_gdf data_dir

In [2]:
# Import necessary packages
import os
import pathlib
from glob import glob

import pandas as pd # Aggregating and data manipulation
import matplotlib.pyplot as plt # Overlay pandas and xarray plots
import rioxarray as rxr # Work with raster data
import xarray as xr

## Load in climate data

For each grassland: Download one climate variable from the MACAv2 THREDDS data server as raster data.

MACA v2 model: CanESM2
* Climate variable: precipitation
* Emissions scenarios being compared: rcp4.5 and rcp8.5
* Time period: 2076 - 2080
* Monthly predictions

In [3]:
# # Make MACA data directory
# maca_85_dir = os.path.join(data_dir, 'maca-85')
# os.makedirs(maca_85_dir, exist_ok=True)
# maca_85_dir

# maca_85_path = os.path.join(maca_85_dir, '.*nc')

# # Only download once
# if not os.path.exists(maca_85_path):
#     maca_85_ds = xr.open_dataset(maca_85_url)
#     maca_85_ds.to_netcdf(maca_85_path)

In [4]:
# Define function to convert longitude/latitude
def convert_longitude(longitude):
    """Convert longitude range from 0-360 t0 -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

maca_da_list = []
for site_name, site_gdf in {
    'Sheyenne Grasslands': sheyenne_grasslands_gdf,
    'Caddo Grasslands': caddo_grasslands_gdf}.items():
    for rcp_value in ['rcp85', 'rcp45']:
        # Define url
        maca_url = (
            'http://thredds.northwestknowledge.net:8080/thredds/dodsC'
            '/MACAV2/CanESM2'
            '/macav2metdata_pr_CanESM2_r1i1p1'
            f'_{rcp_value}'
            '_2076_2080_CONUS'
            '_monthly.nc')

        # Squeeze Dataset
        maca_da = xr.open_dataset(maca_url).squeeze().precipitation

        # Define bounds
        bounds_maca = (site_gdf
                        .to_crs(maca_da.rio.crs)
                        .total_bounds)

        # Change maca_ds longitude values to match the grasslands gdfs
        maca_da = maca_da.assign_coords(
            lon=("lon", [convert_longitude(l) for l in maca_da.lon.values]))

        # Set spatial dimensions of maca_da
        maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

        # Crop maca_da
        maca_da = maca_da.rio.clip_box(*bounds_maca)
        maca_da_list.append(dict(
            site_name=site_name,
            rcp_value=rcp_value,
            da=maca_da))

maca_df = pd.DataFrame(maca_da_list)
maca_df

,site_name,rcp_value,da
0,Sheyenne Grasslands,rcp85,[[[<xarray.DataArray 'precipitation' ()> Size:...
1,Sheyenne Grasslands,rcp45,[[[<xarray.DataArray 'precipitation' ()> Size:...
2,Caddo Grasslands,rcp85,[[[<xarray.DataArray 'precipitation' ()> Size:...
3,Caddo Grasslands,rcp45,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [5]:
maca_da_list[0]

{'site_name': 'Sheyenne Grasslands',
 'rcp_value': 'rcp85',
 'da': <xarray.DataArray 'precipitation' (time: 60, lat: 12, lon: 14)> Size: 40kB
 [10080 values with dtype=float32]
 Coordinates:
   * lat      (lat) float64 96B 46.1 46.15 46.19 46.23 ... 46.48 46.52 46.56
   * time     (time) object 480B 2076-01-15 00:00:00 ... 2080-12-15 00:00:00
   * lon      (lon) float64 112B -97.48 -97.44 -97.4 ... -97.02 -96.98 -96.94
     crs      int64 8B 0
 Attributes:
     long_name:      Monthly Precipitation Amount
     units:          mm
     standard_name:  precipitation
     cell_methods:   time: sum(interval: 24 hours): sum over days
     comments:       Total monthly precipitation at surface: includes both liq...
     _ChunkSizes:    [ 10  44 107]}